# Chrome Built-in AI Demo in JupyterLab and JupyterLite Python Notebook

This notebook demonstrates the Python wrapper for Chrome's built-in AI Prompt API.

https://developer.chrome.com/docs/ai/prompt-api

**Requirements:**
- Chrome browser with Prompt API enabled
- Running in JupyterLab, Jupyter Notebook, or JupyterLite (statically served, all execution locally in browser)

## For developers: Using the Prompt API
* Open Chrome and go to `chrome://flags`.
* Enable the `#prompt-api-for-gemini-nano` flag.
* Enable the `#optimization-guide-on-device-model` flag. If you see a "BypassPerfRequirement" option, select it.
* Restart Chrome.

### &#128679; JupyterLab Only ATM &#128679;
The Python `wiki3_ai` package wrapping the [Built-in AI Prompt API](https://developer.chrome.com/docs/ai/prompt-api) currently only works in JupyterLab notebooks, not JupyterLite.

Launch the https://github.com/wiki3-ai/wiki3-ai-site repo and open the [files/prompt_ai.ipynb](https://github.com/wiki3-ai/wiki3-ai-site/blob/main/files/prompt_ai.ipynb) notebook file (source for this page) using this link:</br>
THIS DOESN'T WORK YET
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/wiki3-ai/wiki3-ai-site/HEAD?urlpath=%2Fdoc%2Ftree%2Ffiles%2Fprompt_ai.ipynb)

Working option for now is GitHub Codespaces:</br>
https://codespaces.new/wiki3-ai/wiki3-ai-site?quickstart=1

In [1]:
%pip install -q anywidget ipywidgets wiki3_ai

Note: you may need to restart the kernel to use updated packages.


In [2]:
%gui asyncio

In [3]:
# Import the library
from wiki3_ai import LanguageModel

In [4]:
LanguageModel.widget()

In [5]:
import wiki3_ai
wiki3_ai.__version__

'0.5.1'

## Check Availability

First, let's check if the Prompt API is available in your browser.

In [6]:
# Check if the API is available
await LanguageModel.availability()

'available'

## Get Model Parameters

Let's see what parameters the model supports.

In [7]:
params = await LanguageModel.params()

if params:
    print(f"Default temperature: {params.get("defaultTemperature")}")
    print(f"Max temperature: {params.get("maxTemperature")}")
    print(f"Default top-K: {params.get("defaultTopK")}")
    print(f"Max top-K: {params.get("maxTopK")}")
else:
    print("Model parameters not available")

params

Default temperature: 1
Max temperature: 2
Default top-K: 3
Max top-K: 128


{'defaultTopK': 3,
 'maxTopK': 128,
 'defaultTemperature': 1,
 'maxTemperature': 2}

## Create LanguageModel Session

In [8]:
lm = await LanguageModel.create()
lm

## Simple Prompt

Create a session and send a simple prompt.

In [9]:
# Send a prompt
result = await lm.prompt("Write a haiku about Python programming.")
print(result)

Clean syntax flows free,
Logic blooms, a vibrant code,
Power in each line. 






## System Prompt

Use a system prompt to set the context for the conversation.

In [10]:
# Create a session with a system prompt
assistant_session = await LanguageModel.create(
    {
        "initialPrompts": [
            {
                "role": "system",
                "content": "You are a helpful Python programming assistant who gives concise answers.",
            }
        ]
    }
)

# Ask a question
result = await assistant_session.prompt("How do I read a CSV file in Python?")
print(result)

```python
import csv

with open('your_file.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)
```

This code reads a CSV file, iterates through each row, and prints it.  Replace `'your_file.csv'` with the actual filename.  `csv.reader` handles the parsing of the comma-separated values.



In [11]:
result = await assistant_session.prompt("Please return just the code to read a CSV file in Python.  Include detailed comments.")
print(result)

```python
import csv

def read_csv(filename):
  """
  Reads a CSV file and returns each row as a list.

  Args:
    filename: The name of the CSV file to read.

  Returns:
    None. Prints each row to the console.
  """
  with open(filename, 'r') as file:  # Open the file in read mode ('r') using a 'with' statement (ensures file closure)
    reader = csv.reader(file)  # Create a CSV reader object
    for row in reader:  # Iterate through each row in the CSV file
      print(row)  # Print each row (which is a list of strings)



# Example usage (assuming you have a file named 'your_file.csv'):
# read_csv('your_file.csv')
```



## Streaming Response

Stream the response as it's generated.

In [12]:
print("Response: ", end="")
async for chunk in assistant_session.prompt_streaming("Tell me a short joke about computers."):
    print(chunk, end="", flush=True)
print()

Response: Why was the computer cold? 

It left its Windows open!






In [13]:
async for chunk in assistant_session.prompt_streaming("Read me a long poem please."):
    print(chunk, end="", flush=True)
print()

Please specify a poem! I am an AI and don't inherently "know" poems. 

**Tell me a poem you'd like me to read to you.** I can then display it for you. 

I can handle many poems, from classics like Shakespeare to modern works. Just paste the text of the poem here, and I'll be happy to "read" it to you. 😊






## Multi-turn Conversation

Have a conversation with context maintained across prompts.

In [14]:
# First message
result1 = await assistant_session.prompt("What is a list comprehension?")
print("Assistant:", result1)
print()

Assistant: ```python
# List comprehension creates a new list based on an existing iterable (like a list, tuple, or range).

# Basic syntax: [expression for item in iterable if condition]

# Example: Create a list of squares of numbers from 0 to 4
squares = [x**2 for x in range(5)]  # squares will be [0, 1, 4, 9, 16]

# Another example:  Filter even numbers from a list
numbers = [1, 2, 3, 4, 5, 6]
even_numbers = [x for x in numbers if x % 2 == 0] # even_numbers will be [2, 4, 6]

# List comprehensions are often more concise and readable than traditional loops for creating lists.
```

In essence, it's a compact way to create lists. It avoids explicit loops, making code more Pythonic. The `if condition` part is optional; it allows you to filter items from the iterable.



In [15]:
# Follow-up message (the assistant remembers the context)
result2 = await assistant_session.prompt("Can you show me an example?")
print("Assistant:", result2)

Assistant: ```python
# Traditional loop to square numbers:
squares_loop = []
for i in range(5):
  squares_loop.append(i**2)
print(f"Squares using loop: {squares_loop}")

# List comprehension to square numbers:
squares_comprehension = [i**2 for i in range(5)]
print(f"Squares using comprehension: {squares_comprehension}")
```

This demonstrates how a list comprehension accomplishes the same task as a `for` loop in a more compact way. The output will be:

```
Squares using loop: [0, 1, 4, 9, 16]
Squares using comprehension: [0, 1, 4, 9, 16]
```

Notice how the list comprehension is more concise and often considered more readable.






## Check Token Usage

Monitor how many tokens you've used.

In [16]:
print(f"Current usage: {assistant_session.input_usage}/{assistant_session.input_quota} tokens")

# Measure how many tokens a prompt would use
usage = await assistant_session.measure_input_usage("What is machine learning?")
print(f"\nThis prompt would use approximately {usage} tokens")

Current usage: 896/9216 tokens

This prompt would use approximately 13 tokens


## Structured Output

Use JSON schema to get structured responses.

In [17]:
import json

# Define a JSON schema
schema = {
    "type": "object",
    "required": ["sentiment", "score"],
    "properties": {
        "sentiment": {"type": "string", "enum": ["positive", "negative", "neutral"]},
        "score": {"type": "number", "minimum": 0, "maximum": 1},
    },
}

# Get structured response
result = await lm.prompt(
    "Analyze this review: The product exceeded all my expectations! Absolutely amazing!",
    {"responseConstraint": schema},
)

# Parse the JSON response
data = json.loads(result)
print(f"Sentiment: {data['sentiment']}")
print(f"Score: {data['score']}")

Sentiment: positive
Score: 0.95


## Session Cloning

Clone a session to create different conversation branches.

In [18]:
# Start a story
story_session = await LanguageModel.create(
    {"initialPrompts": [{"role": "system", "content": "You are a creative storyteller."}]}
)

await story_session.prompt("Once upon a time, there was a dragon.")

# Create two different story branches
branch1 = await story_session.clone()
branch2 = await story_session.clone()

print("=== Branch 1 ===")
async for chunk in branch1.prompt_streaming("The dragon was friendly and helpful."):
    print(chunk, end="", flush=True)
print("=== End of 1 ===")

print("=== Branch 2 ===")
async for chunk in branch2.prompt_streaming("The dragon was fierce and terrifying."):
    print(chunk, end="", flush=True)
print("=== End of 2 ===")

=== Branch 1 ===
Once upon a time, there was a dragon named Zephyr, but he wasn’t like the dragons of legend. He didn't hoard gold or breathe roaring flames that melted mountains. Zephyr hoarded stories, and he breathed shimmering, iridescent bubbles of laughter. His scales were the color of a twilight sky, swirling with hues of amethyst, rose, and deep indigo. And he was exceptionally friendly and helpful.

He lived in a hidden valley nestled amongst whispering willow trees and crystalline streams, a place so secluded that few humans ever stumbled upon it. Instead of terrorizing villages, Zephyr spent his days flitting through the clouds, listening to the whispers of the wind and the songs of birds. He gathered tales from travelers, from children’s dreams, and even from the rustling leaves of ancient forests. 

He'd collect these stories, carefully storing them within the shimmering bubbles he breathed. Each bubble held a whole adventure - a brave knight’s quest, a mischievous pixie’s

## Cleanup

Destroy sessions when you're done to free up resources.

In [19]:
await lm.destroy()
await assistant_session.destroy()
await story_session.destroy()
await branch1.destroy()
await branch2.destroy()

print("✅ All sessions destroyed")

✅ All sessions destroyed
